#Book Recommender System



#Motivation

Books have had a great impact on my life and I believe a good book can totally change the way one perceives the world. Hence creation of a recommendation system for books was something that needed to be worked upon.

Demo link - https://sameerr007-bookrecommendernytbestseller.hf.space/

Takes a book that user has read as a input and gives recommendation based on top 70 NYT bestsellers.

#Creating dataset

In [125]:
#fetching data from neyworktime servers through api
import requests
data_overview=requests.get("https://api.nytimes.com/svc/books/v3/lists/overview.json?api-key=KrShoQ5xJeMfF80AHfZKQzvNPrB2HucR").json()

In [126]:
#creating dataframe for storing the required features of top 90 nyt bestseller books
import pandas as pd
import numpy as np
data = pd.DataFrame(index=range(90),columns=['title','author','genres','description'])

In [128]:
data.head(5)

,title,author,genres,description
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [129]:
#storing the title from json data in the respective created dataframe
k=0
for i in range(18):
  for j in range(5):
    bt=str(data_overview["results"]['lists'][i]['books'][j]['title'])
    data['title'][k]=bt
    k=k+1

In [130]:
data.shape

(90, 4)

In [131]:
data.isnull().sum()

title           0
author         90
genres         90
description    90
dtype: int64

In [132]:
data.head()

,title,author,genres,description
0,ONLY THE DEAD,NaN,NaN,NaN
1,HAPPY PLACE,NaN,NaN,NaN
2,IT ENDS WITH US,NaN,NaN,NaN
3,IT STARTS WITH US,NaN,NaN,NaN
4,LESSONS IN CHEMISTRY,NaN,NaN,NaN


In [133]:
#checking for duplicates in title column
data['title'].duplicated().any()

True

In [134]:
#Droping the duplicate rows
data.drop_duplicates(["title"],inplace=True)

In [135]:
data['title'].duplicated().any()

False

In [136]:
data.shape

(74, 4)

In [137]:
data.head(100)

,title,author,genres,description
0,ONLY THE DEAD,NaN,NaN,NaN
1,HAPPY PLACE,NaN,NaN,NaN
2,IT ENDS WITH US,NaN,NaN,NaN
3,IT STARTS WITH US,NaN,NaN,NaN
4,LESSONS IN CHEMISTRY,NaN,NaN,NaN
...,...,...,...,...
85,IF HE HAD BEEN WITH ME,NaN,NaN,NaN
86,GIRL IN PIECES,NaN,NaN,NaN
87,BETTER THAN THE MOVIES,NaN,NaN,NaN
88,THE WAY I USED TO BE,NaN,NaN,NaN


In [138]:
#to reset indices after droping duplicates 
data.reset_index(inplace=True)

In [139]:
#to change the titles in the form to pass into google books server to extract the information about other features
def gsearch(book):
  a=book.split()
  if len(a)==1:
    return(a[0])
  else:
    abc=""
    for i in a:
      abc=abc+i+"+"
    return(abc[0:len(abc)-1])

In [140]:
data["title_gs"]=data['title'].apply(gsearch)

In [141]:
data.head()

,index,title,author,genres,description,title_gs
0,0,ONLY THE DEAD,NaN,NaN,NaN,ONLY+THE+DEAD
1,1,HAPPY PLACE,NaN,NaN,NaN,HAPPY+PLACE
2,2,IT ENDS WITH US,NaN,NaN,NaN,IT+ENDS+WITH+US
3,3,IT STARTS WITH US,NaN,NaN,NaN,IT+STARTS+WITH+US
4,4,LESSONS IN CHEMISTRY,NaN,NaN,NaN,LESSONS+IN+CHEMISTRY


In [142]:
#add the column for book img links
data['img']=np.nan

In [143]:
data.head()

,index,title,author,genres,description,title_gs,img
0,0,ONLY THE DEAD,NaN,NaN,NaN,ONLY+THE+DEAD,NaN
1,1,HAPPY PLACE,NaN,NaN,NaN,HAPPY+PLACE,NaN
2,2,IT ENDS WITH US,NaN,NaN,NaN,IT+ENDS+WITH+US,NaN
3,3,IT STARTS WITH US,NaN,NaN,NaN,IT+STARTS+WITH+US,NaN
4,4,LESSONS IN CHEMISTRY,NaN,NaN,NaN,LESSONS+IN+CHEMISTRY,NaN


In [144]:
import requests
j=0
for i in data['title_gs']:
    data_id=requests.get("https://www.googleapis.com/books/v1/volumes?q="+i).json()
    try:
      btt=str(data_id['items'][0]['volumeInfo']['title'])
      ba=str(data_id['items'][0]['volumeInfo']['authors'][0])
      data['title'][j]=btt
      data['author'][j]=ba
      data['img'][j]=data_id['items'][0]['volumeInfo']['imageLinks']['thumbnail']
    except:
      pass
    try:
      desc=str(data_id['items'][0]['volumeInfo']['description'])
      data["description"][j]=desc
    except:
      pass
    try:
      data["genres"][j]=str(data_id['items'][0]['volumeInfo']['categories'][0])
    except:
      data["genres"][j]=" "
 
    j=j+1


In [145]:
data.head()

,index,title,author,genres,description,title_gs,img
0,0,Only the Dead,Bear F. Braumoeller,POLITICAL SCIENCE,The idea that war is going out of style has be...,ONLY+THE+DEAD,http://books.google.com/books/content?id=viaeD...
1,1,Happy Place,Emily Henry,Fiction,Harriet and Wyn are the perfect couple - they ...,HAPPY+PLACE,http://books.google.com/books/content?id=-AxSz...
2,2,It Starts with Us,Colleen Hoover,Fiction,"Lily and her ex-husband, Ryle, have just settl...",IT+ENDS+WITH+US,http://books.google.com/books/content?id=CMeVE...
3,3,It Starts with Us,Colleen Hoover,Fiction,"Lily and her ex-husband, Ryle, have just settl...",IT+STARTS+WITH+US,http://books.google.com/books/content?id=CMeVE...
4,4,Lessons in Chemistry,Bonnie Garmus,Fiction,As read on BBC Radio 4 Book at Bedtime THE #1 ...,LESSONS+IN+CHEMISTRY,http://books.google.com/books/content?id=G2U4E...


In [146]:
data.isnull().sum()

index          0
title          0
author         0
genres         0
description    1
title_gs       0
img            1
dtype: int64

In [147]:
data.dropna(inplace=True)

In [148]:
data.shape

(72, 7)

In [149]:
data.head()

,index,title,author,genres,description,title_gs,img
0,0,Only the Dead,Bear F. Braumoeller,POLITICAL SCIENCE,The idea that war is going out of style has be...,ONLY+THE+DEAD,http://books.google.com/books/content?id=viaeD...
1,1,Happy Place,Emily Henry,Fiction,Harriet and Wyn are the perfect couple - they ...,HAPPY+PLACE,http://books.google.com/books/content?id=-AxSz...
2,2,It Starts with Us,Colleen Hoover,Fiction,"Lily and her ex-husband, Ryle, have just settl...",IT+ENDS+WITH+US,http://books.google.com/books/content?id=CMeVE...
3,3,It Starts with Us,Colleen Hoover,Fiction,"Lily and her ex-husband, Ryle, have just settl...",IT+STARTS+WITH+US,http://books.google.com/books/content?id=CMeVE...
4,4,Lessons in Chemistry,Bonnie Garmus,Fiction,As read on BBC Radio 4 Book at Bedtime THE #1 ...,LESSONS+IN+CHEMISTRY,http://books.google.com/books/content?id=G2U4E...


#Feature Engineering

In [150]:
#to remove spaces in between author names to treat each name as a unique entity
def remove(s):
  s=str(s)
  return(s.replace(" ",""))

In [151]:
data["author"]=data["author"].apply(remove)

In [152]:
data["author"]

0     BearF.Braumoeller
1            EmilyHenry
2         ColleenHoover
3         ColleenHoover
4          BonnieGarmus
            ...        
69          LauraNowlin
70      KathleenGlasgow
71          LynnPainter
72           AmberSmith
73       KarenM.McManus
Name: author, Length: 72, dtype: object

In [153]:
data["genres"]=data["genres"].apply(remove)

In [154]:
#combining all the features that uniquely defines the book into one column
combined_features = data['genres']+' '+data['author']+' '+data['description']

In [155]:
combined_features

0     POLITICALSCIENCE BearF.Braumoeller The idea th...
1     Fiction EmilyHenry Harriet and Wyn are the per...
2     Fiction ColleenHoover Lily and her ex-husband,...
3     Fiction ColleenHoover Lily and her ex-husband,...
4     Fiction BonnieGarmus As read on BBC Radio 4 Bo...
                            ...                        
69    JuvenileFiction LauraNowlin If he had been wit...
70    YoungAdultFiction KathleenGlasgow #1 NEW YORK ...
71    JuvenileFiction LynnPainter A USA TODAY and Ne...
72    YoungAdultFiction AmberSmith "After fourteen-y...
73    YoungAdultFiction KarenM.McManus The internati...
Length: 72, dtype: object

In [156]:
#to convert all the features into lowercase to remove any bias due to the case
def low(a):
    return(a.lower())

In [157]:
combined_features=combined_features.apply(low)
combined_features

0     politicalscience bearf.braumoeller the idea th...
1     fiction emilyhenry harriet and wyn are the per...
2     fiction colleenhoover lily and her ex-husband,...
3     fiction colleenhoover lily and her ex-husband,...
4     fiction bonniegarmus as read on bbc radio 4 bo...
                            ...                        
69    juvenilefiction lauranowlin if he had been wit...
70    youngadultfiction kathleenglasgow #1 new york ...
71    juvenilefiction lynnpainter a usa today and ne...
72    youngadultfiction ambersmith "after fourteen-y...
73    youngadultfiction karenm.mcmanus the internati...
Length: 72, dtype: object

In [158]:
data['tags']=combined_features

In [159]:
data['tags'].apply(len).median()

1168.5

In [160]:
#to use porter stemmer to change each word into its root word
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [161]:
def stem(a):
    y=[]
    for i in a.split():
        y.append(ps.stem(i))
    return(" ".join(y))

In [162]:
data['tags']=data['tags'].apply(stem)

#Model Development

In [163]:
#to use count Vectorizer to encode the words so that our model can use it
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=10000,stop_words='english')

In [164]:
vec=cv.fit_transform(data['tags'])

In [165]:
len(cv.get_feature_names_out())

3270

In [166]:
#to implement cosine similarity on the encoded tags so as to compute the similarity between each of the books
from sklearn.metrics.pairwise import cosine_similarity
print(vec.shape)
similarity=cosine_similarity(vec)
print(similarity.shape)

(72, 3270)
(72, 72)


#User input commands

In [167]:
def recommend(book):
    gs=gsearch(book)
    data_id=requests.get("https://www.googleapis.com/books/v1/volumes?q="+gs).json()    
    booka=ba=str(data_id['items'][0]['volumeInfo']['authors'][0])
    desc=str(data_id['items'][0]['volumeInfo']['description'])
    genres=str(data_id['items'][0]['volumeInfo']['categories'][0])
    tag=genres+" "+booka+" "+desc
    tag=tag.lower()
    tag=stem(tag)
    vec1=cv.transform([tag])
    from scipy.sparse import vstack
    fvec=vstack([vec1, vec])
    similarity=cosine_similarity(fvec)
    distances=similarity[0]
    #print(distances)
    print(sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1]))
    book_list_index=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1]
    if (str(data.iloc[book_list_index[0]-1]['title']).lower())==(book.lower()):
      #if the book user has typed is already in the database so we output the second most(not the first most) similar book
      print("**")
      book_list_index=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[2]
      print("Book recomended-"+data.iloc[book_list_index[0]-1]['title'])
      print("Similarity percentage="+str(round(book_list_index[1]*100,2)))
      print("Synopsis")
      print(data.iloc[book_list_index[0]-1]['description'])
    else:
      print("Book recomended-"+data.iloc[book_list_index[0]-1]['title'])
      print("Similarity percentage="+str(round(book_list_index[1]*100,2)))
      print("Synopsis")
      print(data.iloc[book_list_index[0]-1]['description'])

In [168]:
book_name = input(' Enter book name : ')
recommend(book_name)

 Enter book name : The kite runner
[(0, 0.9999999999999997), (66, 0.2405351177211819), (28, 0.23301366225045), (44, 0.21747941810042), (62, 0.21630645583290953), (46, 0.20619652471058064), (13, 0.2029385157727364), (67, 0.17601439465465724), (61, 0.16594796272900444), (63, 0.1564028117724636), (29, 0.1527830828380352), (45, 0.1525478508542764), (53, 0.15179418517972906), (25, 0.1517797003106489), (71, 0.1515228816828316), (65, 0.14638501094227996), (39, 0.1440046082211958), (50, 0.1354460635337592), (38, 0.131929695183502), (16, 0.13173888881045848), (55, 0.1297186304156927), (34, 0.12930363550315668), (15, 0.12407291674636628), (36, 0.12298800925361815), (41, 0.12229989761755711), (60, 0.11664236870396086), (58, 0.11572751247156894), (69, 0.11470786693528087), (57, 0.11214032600406415), (49, 0.11135885079684349), (59, 0.11048143863155316), (51, 0.11024909044875568), (24, 0.10765409408615137), (3, 0.10482848367219183), (4, 0.10482848367219183), (5, 0.09936414126574183), (43, 0.09652341

#Deployment into a server


The code has been deployed in the streamlit server with the help of the hugging face spaces

Repository link for deployment is - https://huggingface.co/spaces/SameerR007/BookrecommenderNYTbestseller/tree/main

Demo link is - https://sameerr007-bookrecommendernytbestseller.hf.space/ 